# Trumon API

In [1]:
!export CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"

In [2]:
# hanya untuk ujicoba

CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"

# ALTER TABLE Transaksi MODIFY CustomField3 TEXT;

In [3]:
 ! pip install sqlalchemy mysql-connector-python fastapi nest_asyncio uvicorn pyngrok

# remove all Not Needed

In [4]:
import unicodedata, re, itertools, sys

all_chars = (chr(i) for i in range(sys.maxunicode))
categories = {'Cc'}
control_chars = ''.join(c for c in all_chars if unicodedata.category(c) in categories)
# or equivalently and much more efficiently
control_chars = ''.join(map(chr, itertools.chain(range(0x00,0x20), range(0x7f,0xa0))))

control_char_re = re.compile('[%s]' % re.escape(control_chars))

def remove_control_chars(s):
    return control_char_re.sub('', s)



# Methode to Parser

In [5]:
import re

class Parser:
    
    
    No_mor=None # nomer
    tgl = None # tanggal
    jam_brong=None  # jam dari recipt
    NDP =None # Nilai dan Pajak
    subtotal=None # subtotal belum kena pajak
    pajak=None # Pajak
    tipe=None # take away
    payment=None # tipe payment
    data=None
    RcpSts=None  # Reciept status
    SKU=None 
    data2=None
    Tipe=None
    def __init__(self, data):
        self.data=data.decode('utf-8','ignore').encode("utf-8").decode('utf-8') # this data
        self.data2=str(data)
        self.getNo_mor()
        self.getPayment()
        self.getNDP()
        self.getTgl()
        self.getSubTotal()
        self.getJam_brong()
        self.getPajak()
        self.getRcpSts()
        self.getTipe()
        self.getSKU()
        
    def getNo_mor(self):
        subRegex = re.compile(r'(Rcpt#:[^\d]+\d+[^\d]+\d+[^\d]+\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.No_mor=remove_control_chars(mo.group(1))
        else:
            self.No_mor = ""
        
    def getNDP(self): 
        ndpRegex = re.compile(r'[^B]TOTAL[^\d]+(\d+\,\d+)')
        mo = ndpRegex.search(self.data)
        if(mo):
            #print('total: ' + mo.group(1))
            self.NDP=mo.group(1)
        else:
            self.NDP=" "
            # nilai dan pajak disebut sebagai total
        
    def getTgl(self):
        subRegex = re.compile(r'(\d+\/[^\d]+\d{2}\/[^\d]+\d{4})')
        mo = subRegex.search(self.data)
        
        if(re.compile(r'(\d+\/\d+\/\d+)').search(self.data)):
            self.tgl=remove_control_chars(re.compile(r'(\d+\/\d+\/\d+)').search(self.data).group(1).encode("ascii", "ignore").decode().replace("//", "-"))
        elif(mo):
            self.tgl=remove_control_chars(mo.group(1).encode("ascii", "ignore").decode().replace("//", "-"))
        
        else:
            self.tgl = ""
            
    def getSubTotal(self):
        subRegex = re.compile(r'SUBTOTAL[^\d]+(\d+\,\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.subtotal=mo.group(1)
        elif(re.compile(r'Closed Bill(\d+,\d+)').search(self.data)):
            self.subtotal=re.compile(r'Closed Bill(\d+,\d+)').search(self.data).group(1)
        else:
            self.subtotal=" "
        
    def getJam_brong(self):
        subRegex = re.compile(r'(\d{2}\:\d{2})')
        mo = subRegex.search(self.data)
        if(mo):
            self.jam_brong=mo.group(1)
        else:
            self.jam_brong=" "
        
    def getPajak(self):
        subRegex = re.compile(r'10%[^\d]+(\d+\,\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.pajak=mo.group(1)
        else:
            self.pajak=" "
        
    def getRcpSts(self):
        if(re.compile(r'(This Is PreSettlement Rcpt)').search(self.data)):
            self.RcpSts=re.compile(r'(This Is PreSettlement Rcpt)').search(self.data).group(1)
        elif(re.compile(r'(Zero Sales)').search(self.data)):
            self.RcpSts=re.compile(r'(Zero Sales)').search(self.data).group(1)
        else:   
            #Zero Sales
            self.RcpSts=""
            
    def getTipe(self):
        subRegex = re.compile(r'(TAKE AWAY)')
        mo = subRegex.search(self.data)
        if(re.compile(r'(Sales Day Report)').search(self.data)):
            self.Tipe=re.compile(r'(Sales Day Report)').search(self.data).group(1)
        elif(re.compile(r'(TAKE AWAY)').search(self.data)):
            self.Tipe=re.compile(r'(TAKE AWAY)').search(self.data).group(1)
        elif(re.compile(r'(DINE IN)').search(self.data)):
            self.Tipe=re.compile(r'(DINE IN)').search(self.data).group(1)
        else:
            self.Tipe=""
            
    def getPayment(self):
        subRegex = re.compile(r'BNI')
        mo = subRegex.search(self.data)
        if(re.compile(r'(BRI CREDIT CARD)').search(self.data)):
            self.payment=re.compile(r'(BRI CREDIT CARD)').search(self.data).group(1)
        elif(re.compile(r'(GO-PAY)').search(self.data)):
            self.payment=re.compile(r'(GO-PAY)').search(self.data).group(1)
        elif(re.compile(r'(SHOPEE)').search(self.data)):
            self.payment=re.compile(r'(SHOPEE)').search(self.data).group(1)
        elif(re.compile(r'(BCA DEBIT)').search(self.data)):
            self.payment=re.compile(r'(BCA DEBIT)').search(self.data).group(1)
        elif(re.compile(r'(GO-RESTO)').search(self.data)):
            self.payment=re.compile(r'(GO-RESTO)').search(self.data).group(1)
        elif(re.compile(r'(GRABFOOD)').search(self.data)):
            self.payment=re.compile(r'(GRABFOOD)').search(self.data).group(1)
        else:
            self.payment=""
    
    def getSKU(self):
        # (?<=TAKE AWAY)(.*)(?=SUBTOTAL)
        if(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data)):
            self.SKU=remove_control_chars(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data).group(1))
        elif(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data)):
            self.SKU=remove_control_chars(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data).group(1))
        elif(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data2)):
            self.SKU=remove_control_chars(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data2).group(1))
        elif(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data2)):
            self.SKU=remove_control_chars(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data2).group(1))
        else:
            self.SKU=""

# Ini bagian service API

In [6]:
#


from fastapi import FastAPI
from starlette.middleware import Middleware
#from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware.cors import CORSMiddleware

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
import datetime
import os
import sqlalchemy as db
from sqlalchemy import Column, ForeignKey, Integer, String, Numeric, DateTime, ForeignKey, CHAR, Table 

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [7]:
#CONNECTIONSTRING = os.getenv('CONNECTIONSTRING')
CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"
engine = create_engine(CONNECTIONSTRING)

Base = declarative_base(engine)

Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

connection = engine.connect()
metadata = db.MetaData()

class Transaksi(Base):
    __tablename__ = "Transaksi"
    __table_args__ = {"autoload": True}
    
class Mentah(Base):
    __tablename__="FileTransferStage2"
    __table_args__ = {"autoload": True}

class DeviceTable(Base):
    __tablename__ = "DeviceTable"
    __table_args__ = {"autoload": True}
    
class Alarm(Base):
    __tablename__ = "Alarm"
    __table_args__ = {"autoload": True}
    


In [8]:
def getParserMode(devid):
    #
    # untuk mendapatkan mode parser
    #
    getFlag = session.query(DeviceTable).filter(DeviceTable.DeviceId == devid).one()
    return getFlag.ParserMode

In [9]:
def checkJenisRcp(nmr, tgl, valnil):
    
    jumlah = session.query(Transaksi).filter(Transaksi.Nomor==nmr).filter(Transaksi.Tanggal==tgl).filter(Transaksi.Nilai==valnil).count()
    return jumlah

# Insert to Transaction

In [10]:
def InsertToTransaction(idFileTransferStage2, DevId, FT, No_mor,
                        tgl, Jam_brong, subtotal, Pjk, NDP, payment, Tipe, SKU, RcpSts, JnsRcp):
    
    # getTrans = session.query(Transaksi).filter(Transaksi.DeviceId == "mantap-mantap").one()
    trans_insert =  Transaksi(RefSN=idFileTransferStage2, DeviceId=DevId, 
                              FileTime=FT, Nomor=No_mor, Tanggal=tgl, Jam=Jam_brong, Nilai=subtotal,
                              Pajak=Pjk, NilaiDanPajak=NDP, CustomField1=payment, CustomField2=Tipe, 
                              CustomField3=SKU, RcpSTS=RcpSts, JenisRcp=JnsRcp)
    session.add(trans_insert)
    session.commit()

# Update Flag
    

In [11]:
#def updateFlag():
def updateFT2Flag(idFTS2, flag):
    FT2_update = session.query(Mentah).filter(Mentah.idFileTransferStage2 == idFTS2).first()
    FT2_update.FlagParser=flag
    session.commit()
    return FT2_update.FlagParser

In [12]:
alarmCode={
    10: "OS FAILED",
    9: "PUSH BUTTON PRESSED",
    8: "POWER MANAGEMENT SENSOR FAILED",
    7: "FAILED TO ACCESS MICROSD",
    6: "BATTERY UNDETECTED",
    5: "POWER CABLE UNPLUGGED",
    4: "TEMPERATURE UPPER LIMIT",
    3: "TAMPER 2: ORIENTATION CHANGED",
    2: "TAMPER 1: SHAKE/FREEFALL",
    1: "PRINTER CABLE UNPLUGGED",
    133: "Power Cable Unplugged Release",
    134: "Battery Undetected Release",
    129: "Printer Cable Unplugged Release",
    130: "Tamper 1: Shake/Freefall Release",
    131: "PRINTER CABLE UNPLUGGED",
    132: "Temperature Upper Limit Release",
    135: "Failed to Access MicroSD Release",
    136: "Power Management Sensor Failed Release",
    137: "Push Button Pressed Release"
}

In [ ]:


middleware = [
    Middleware(CORSMiddleware,
               allow_origins=["*"],
               allow_methods=["*"], # Allows all methods
               allow_headers=["*"], # Allows all headers
              )
]

app = FastAPI(
    middleware=middleware,
    title="Trumon Api",
    description="Just trumon simple api",
    
)
origins = [
    "*",
]





@app.get('/transaksi')
async def GetTrans(DevID: str = "ALMAR0000003"):

    transList = []

    try:
        transaksis = session.query(Transaksi).all()
        if(transaksis==None):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in transaksis:
                #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
                #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
                #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]

                trans = {
                    "SeqNum": item.SeqNum,
                    "DeviceId": item.DeviceId,
                    "RefSN": item.RefSN,
                    "FileTime": item.FileTime,
                    "Nomor": item.Nomor,
                    "Tanggal": item.Tanggal,
                    "Jam": item.Jam,
                    "Nilai": item.Nilai,
                    "Pajak": item.Pajak,
                    "NilaiDanPajak": item.NilaiDanPajak,
                    "Payment": item.CustomField1,
                    "Tipe": item.CustomField2,
                    "SKU": item.CustomField3,
                    "RcpSts": item.RcpSTS,
                    "JenisRcp": item.JenisRcp,
                    "FlagTransfer": item.FlagTransfer,
                }

                if(item.DeviceId==DevID):
                    transList.append(trans)
                return {"data": transList}
            #print(transList)
    except:
        session.rollback()
        raise

    
@app.get('/permak')
async def Permak(day_resi: str = "05-05-2021", JenisRcp: str="Utama"):

    transList = []

    try:
        transaksis = session.query(Transaksi).filter(Transaksi.Tanggal==day_resi).filter(Transaksi.JenisRcp=="Utama").all()
        if(transaksis==None):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in transaksis:
                #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
                #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
                #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]

                trans = {
                    "SeqNum": item.SeqNum,
                    "DeviceId": item.DeviceId,
                    "RefSN": item.RefSN,
                    "FileTime": item.FileTime,
                    "Nomor": item.Nomor,
                    "Tanggal": item.Tanggal,
                    "Jam": item.Jam,
                    "Nilai": item.Nilai,
                    "Pajak": item.Pajak,
                    "NilaiDanPajak": item.NilaiDanPajak,
                    "Payment": item.CustomField1,
                    "Tipe": item.CustomField2,
                    "SKU": item.CustomField3,
                    "RcpSts": item.RcpSTS,
                    "JenisRcp": item.JenisRcp,
                    "FlagTransfer": item.FlagTransfer,
                }
                
                
                if(item.Nilai!=" " and item.Pajak!=" " and item.NilaiDanPajak!=" " and item.Nomor!=""):
                    transList.append(trans)
                    
                
                
            

            return {"data": transList}
            #print(transList)
    except:
        session.rollback()
        raise

@app.get('/day_resi')
async def Day_resi(day_resi: str = "05-05-2021"):

    transList = []

    try:
        transaksis = session.query(Transaksi).filter(Transaksi.Tanggal==day_resi).all()
        if(transaksis==None):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in transaksis:
                #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
                #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
                #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]

                trans = {
                    "SeqNum": item.SeqNum,
                    "DeviceId": item.DeviceId,
                    "RefSN": item.RefSN,
                    "FileTime": item.FileTime,
                    "Nomor": item.Nomor,
                    "Tanggal": item.Tanggal,
                    "Jam": item.Jam,
                    "Nilai": item.Nilai,
                    "Pajak": item.Pajak,
                    "NilaiDanPajak": item.NilaiDanPajak,
                    "Payment": item.CustomField1,
                    "Tipe": item.CustomField2,
                    "SKU": item.CustomField3,
                    "RcpSts": item.RcpSTS,
                    "JenisRcp": item.JenisRcp,
                    "FlagTransfer": item.FlagTransfer,
                }
                
                transList.append(trans)
                
            

            return {"data": transList}
            #print(transList)
    except:
        session.rollback()
        raise

    


@app.get('/parse')
async def letParse(DevID: str = "ALMAR0000003"):
    
    try:
        if(getParserMode(DevID)==3):
            #session.refresh(Mentah)
            FT2 = session.query(Mentah).filter(db.or_(db.and_(Mentah.DeviceId == DevID, Mentah.FlagParser ==0),db.and_(Mentah.DeviceId == DevID, Mentah.FlagParser ==9999))).all()

            count=0
            for item in FT2:
                # parse 
                p1=Parser(item.FileData)
                # insert 
                if checkJenisRcp(p1.No_mor, p1.tgl, p1.subtotal)>0:
                    JenisRcp="Reprint"
                else:
                    JenisRcp="Utama"
                InsertToTransaction(item.idFileTransferStage2, item.DeviceId, item.FileTime, p1.No_mor,
                                    p1.tgl, p1.jam_brong, p1.subtotal, p1.pajak, p1.NDP, 
                                    p1.payment, p1.Tipe, p1.SKU, p1.RcpSts, JenisRcp)
                # update Flag
                updateFT2Flag(item.idFileTransferStage2, 1)
                count=count+1

            return {"parsed": count}
            session.close()
        else:
            return {"status": "Mode Parser "+DevID+" bukan 3."}
        #print(transList)
    except:
        session.rollback()
        raise
        
    
@app.get('/parse_reset')
async def delParse(DevID: str = "ALMAR0000003"):
    try:
        # delete transaksi
        delTransaksi = session.query(Transaksi).filter(Transaksi.DeviceId==DevID).delete()
        
        # update flag
        FT2 = session.query(Mentah).filter(db.and_(Mentah.DeviceId == DevID)).all()

        count=0
        for item in FT2:
            
            # update Flag
            updateFT2Flag(item.idFileTransferStage2, 0)
            count=count+1
                
        return {"reset_total": count}
    except:
        session.rollback()
        raise

@app.get('/mentah')
async def getMentah(DevID: str = "ALMAR0000003"):
    dataMentahs = []
    try:
        mentahs = session.query(Mentah).all()
        
        if (mentahs==0):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in mentahs:
                #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
                #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
                #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]

                # convert fileData to string
                stringData = str(item.FileData)

                trans = {
                    "idFileTransferStage2": item.idFileTransferStage2,
                    "data": stringData,
                    "filetime": item.FileTime,
                    "flagparser": item.FlagParser
                }

                if(item.DeviceId==DevID):
                    dataMentahs.append(trans)
            
            return {"data": dataMentahs}
            #print(transList)
    except:
        session.rollback()
        raise
    
    


@app.get('/check')
async def checkParser(DevID: str = "ALMAR0000003", RefSN: int=2685):

    transList = []

    try:
        item = session.query(Transaksi).filter(Transaksi.RefSN==RefSN).one()
        mentah = session.query(Mentah).filter(Mentah.idFileTransferStage2==RefSN).one()
        trans = {
            "SeqNum": item.SeqNum,
            "DeviceId": item.DeviceId,
            "RefSN": item.RefSN,
            "FileTime": item.FileTime,
            "Nomor": item.Nomor,
            "Tanggal": item.Tanggal,
            "Jam": item.Jam,
            "Nilai": item.Nilai,
            "Pajak": item.Pajak,
            "NilaiDanPajak": item.NilaiDanPajak,
            "Payment": item.CustomField1,
            "Tipe": item.CustomField2,
            "SKU": item.CustomField3,
            "RcpSts": item.RcpSTS,
            "JenisRcp": item.JenisRcp,
            "FlagTransfer": item.FlagTransfer,
            "Data": str(mentah.FileData),
        }

           
        #print(transList)
    except:
        session.rollback()
        raise
    return {"data": trans}
#app.add_middleware(
#    CORSMiddleware,
#    allow_origins=origins,
#    allow_credentials=True,
#    allow_methods=["*"],
#    allow_headers=["*"],
#)

@app.get('/alarm')
async def getAlarm(start_date: str = "2021-05-05", end_date: str="2021-05-06"):
    alarmList=[]
    try:
        alarms = session.query(Alarm).filter(Alarm.AlarmTime>=start_date).filter(Alarm.AlarmTime<end_date).all()
        if(alarms==None):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in alarms:
                alarm = {
                    "idAlarm": item.idAlarm,
                    "DeviceId": item.DeviceId,
                    "AlarmType": item.AlarmType,
                    "AlarmDesc": alarmCode[item.AlarmType],
                    "AlarmTime": item.AlarmTime
                }
                
                alarmList.append(alarm)
                
        return {"data": alarmList}
    except:
        session.rollback()
        raise
        
@app.get('/latestAlarm')
async def latestAlarm(DevID: str = "ALMAR0000003"):
    alarmList=[]
    try:
        alarms = session.query(Alarm).filter(Alarm.DeviceId==DevID).order_by(Alarm.idAlarm.desc()).limit(10).all()
        if(alarms==None):
            return {"status": "data belum tersedia", "kode": 69}
        else:
            for item in alarms:
                alarm = {
                    "idAlarm": item.idAlarm,
                    "DeviceId": item.DeviceId,
                    "AlarmType": item.AlarmType,
                    "AlarmDesc": alarmCode[item.AlarmType],
                    "AlarmTime": item.AlarmTime
                }
                
                alarmList.append(alarm)
                
        return {"data": alarmList}
    except:
        session.rollback()
        raise
# ngrok tunnel cuma buat akses pake tunnel ngrok 
# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(app, port=8000, host='0.0.0.0', log_level="info", ssl_keyfile="almar.id/privkey.pem", ssl_certfile="almar.id/cert.pem")
#uvicorn.run(app, port=8000, host='0.0.0.0', log_level="info")

INFO:     Started server process [13587]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     36.72.215.85:2452 - "GET /docs HTTP/1.1" 200 OK
INFO:     36.72.215.85:2452 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     36.72.215.85:2452 - "GET /day_resi?day_resi=05-05-2021 HTTP/1.1" 200 OK
INFO:     36.72.215.85:21681 - "GET /permak?day_resi=05-05-2021&JenisRcp=Utama HTTP/1.1" 200 OK
INFO:     180.244.171.101:29073 - "GET /permak?day_resi=05-05-2021&JenisRcp=Utama HTTP/1.1" 200 OK
INFO:     180.244.171.101:29073 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     36.72.215.85:32407 - "GET /day_resi?day_resi=05-05-2021 HTTP/1.1" 200 OK
INFO:     180.244.171.101:23125 - "GET /permak?day_resi=05-05-2021&JenisRcp=Utama HTTP/1.1" 200 OK


In [14]:
!ls faoziaziz.id

README	cert.pem  chain.pem  fullchain.pem  privkey.pem
